In [1]:
from data import get_data
import numpy as np
import pandas as pd
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [9]:
def parkinson(data: pd.DataFrame):
        """
        Calculates the parkinson volatility estimator.
        :return:
        """
        const = 4 * np.log(2)

        rs = (np.log(data["high"] / data["low"]) ** 2.0).rolling(window=30).sum()

        result = (rs / (const * 252)) ** 0.5
        
        return result

In [6]:
aapl = get_data("AAPL", period="2y")

The size of  AAPL is  (501, 7)


In [7]:
aapl

,open,high,low,close,volume,dom,returns
datetime,,,,,,,
2022-06-07 04:00:00+00:00,142.735147,147.333121,142.487943,147.046371,67808200,7,0.017586
2022-06-08 04:00:00+00:00,146.917838,148.193400,145.810373,146.304779,53950200,8,-0.005043
2022-06-09 04:00:00+00:00,145.434613,146.294875,140.935511,141.044281,69473000,9,-0.035956
2022-06-10 04:00:00+00:00,138.710656,139.185282,135.526677,135.595901,91437900,10,-0.038629
2022-06-13 04:00:00+00:00,131.383563,133.687499,129.969568,130.404648,122207100,13,-0.038285
...,...,...,...,...,...,...,...
2024-05-29 04:00:00+00:00,189.610001,192.250000,189.509995,190.289993,53068000,29,0.001579
2024-05-30 04:00:00+00:00,190.759995,192.179993,190.630005,191.289993,49947900,30,0.005255
2024-05-31 04:00:00+00:00,191.440002,192.570007,189.910004,192.250000,75158300,31,0.005019


In [10]:
volatility = parkinson(aapl)

In [12]:
volatility.tail(10)

datetime
2024-05-21 04:00:00+00:00    0.003878
2024-05-22 04:00:00+00:00    0.003886
2024-05-23 04:00:00+00:00    0.003644
2024-05-24 04:00:00+00:00    0.003570
2024-05-28 04:00:00+00:00    0.003541
2024-05-29 04:00:00+00:00    0.003370
2024-05-30 04:00:00+00:00    0.003332
2024-05-31 04:00:00+00:00    0.003340
2024-06-03 04:00:00+00:00    0.003333
2024-06-04 04:00:00+00:00    0.003314
dtype: float64

In [13]:
volatility.size

501